In [2]:
import random
import numpy as np
import sys
from random import randint
from data_load import *
from cwsabie_inner import *
from transE_label import *
from gensim.corpora import Dictionary
from collections import Counter
from gensim.models import Word2Vec
%load_ext cython

# Wsabie

In [3]:
%%cython
import numpy as np
from TypeHierarchy import *
cimport numpy as np
from random import randint
import sys
import cython
cdef extern from "math.h":
    double sqrt(double m)
import math
from libc.stdlib cimport malloc, free

from libc.math cimport exp
from libc.math cimport log
from gensim.matutils import argsort

from libc.string cimport memset
import random
# scipy <= 0.15

import scipy.linalg.blas as fblas
ctypedef np.float32_t REAL_t
cdef int ONE = 1


REAL = np.float32
cdef extern from "/Users/mayk/working/figer/baseline/PLE/Model/warp/voidptr.h":
    void* PyCObject_AsVoidPtr(object obj)
DEF MAX_SENTENCE_LEN = 10000
ctypedef void (*scopy_ptr) (const int *N, const float *X, const int *incX, float *Y, const int *incY) nogil
ctypedef void (*saxpy_ptr) (const int *N, const float *alpha, const float *X, const int *incX, float *Y, const int *incY) nogil
ctypedef float (*sdot_ptr) (const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil
ctypedef double (*dsdot_ptr) (const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil
ctypedef double (*snrm2_ptr) (const int *N, const float *X, const int *incX) nogil
ctypedef void (*sscal_ptr) (const int *N, const float *alpha, const float *X, const int *incX) nogil


cdef scopy_ptr scopy = <scopy_ptr>PyCObject_AsVoidPtr(fblas.scopy._cpointer)  # y = x
cdef saxpy_ptr saxpy=<saxpy_ptr>PyCObject_AsVoidPtr(fblas.saxpy._cpointer)  # y += alpha * x
cdef sdot_ptr sdot=<sdot_ptr>PyCObject_AsVoidPtr(fblas.sdot._cpointer)  # float = dot(x, y)
cdef dsdot_ptr dsdot=<dsdot_ptr>PyCObject_AsVoidPtr(fblas.sdot._cpointer)  # double = dot(x, y)
cdef snrm2_ptr snrm2=<snrm2_ptr>PyCObject_AsVoidPtr(fblas.snrm2._cpointer)  # sqrt(x^2)
cdef sscal_ptr sscal=<sscal_ptr>PyCObject_AsVoidPtr(fblas.sscal._cpointer) # x = alpha * x
DEF EXP_TABLE_SIZE = 10000
DEF MAX_EXP = 50

cdef REAL_t[EXP_TABLE_SIZE] EXP_TABLE
cdef REAL_t[EXP_TABLE_SIZE] LOG_TABLE

cdef REAL_t ONEF = <REAL_t>1.0

# for when fblas.sdot returns a double
cdef REAL_t our_dot_double(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    return <REAL_t>dsdot(N, X, incX, Y, incY)

# for when fblas.sdot returns a float
cdef REAL_t our_dot_float(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    return <REAL_t>sdot(N, X, incX, Y, incY)

# for when no blas available
cdef REAL_t our_dot_noblas(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    # not a true full dot()-implementation: just enough for our cases
    cdef int i
    cdef REAL_t a
    a = <REAL_t>0.0
    for i from 0 <= i < 50 by 1:
        a += X[i] * Y[i]
    return a

# for when no blas available
cdef void our_saxpy_noblas(const int *N, const float *alpha, const float *X, const int *incX, float *Y, const int *incY) nogil:
    cdef int i
    for i from 0 <= i < N[0] by 1:
        Y[i * (incY[0])] = (alpha[0]) * X[i * (incX[0])] + Y[i * (incY[0])]
cdef REAL_t cvdot(vec1,vec2,size):
    cdef int csize = size
    f= dsdot(&csize,<REAL_t *>(np.PyArray_DATA(vec1)),&ONE,<REAL_t *>(np.PyArray_DATA(vec2)),&ONE)
    return f
def csaxpy(vec1,vec2,alpha,size):
    cdef int csize = size
    cdef float calpha = alpha
    f= our_saxpy_noblas(&csize,&calpha,<REAL_t *>(np.PyArray_DATA(vec1)),&ONE,<REAL_t *>(np.PyArray_DATA(vec2)),&ONE)
    return f
cdef REAL_t crank(int k):
    cdef REAL_t loss = 0.
    cdef int i = 1
    for i in range(1,k+1):
        loss += ONEF/i
    return loss
cdef REAL_t rev_crank(int k):
    cdef REAL_t loss = 2.
    cdef int i = 1
    for i in range(1,k+1):
        loss -= ONEF/i
    return loss
cdef REAL_t vsum(REAL_t *vec,int *size):
    cdef int i
    cdef REAL_t product
    product = <REAL_t>0.0
    for i from 0 <= i < size[0] by 1:
        product += vec[i] **2
    return sqrt(product)
def cnorm(vec):
    cdef int size
    size  = len(vec)
    return vsum(<REAL_t *>(np.PyArray_DATA(vec)),&size)
def init():
    for i in range(EXP_TABLE_SIZE):
        EXP_TABLE[i] = <REAL_t>exp((i / <REAL_t>EXP_TABLE_SIZE * 2 - 1) * MAX_EXP)
        EXP_TABLE[i] = <REAL_t>(EXP_TABLE[i] / (EXP_TABLE[i] + 1))
#init()


def ctrain(A,B,insts,size,lr,gradient,it,Verbose=False):
    cdef float error
    next_random = 1
    error = 0.
   # cdef lam = 0.01
    for i,inst in enumerate(insts):
        err,next_random =gradient(A,B,inst,size,next_random,lr=lr)
      
        error += err
        if i % 1000 ==0 and Verbose:
            sys.stdout.write("\rIteration %d " % (it)+ "trained {0:.0f}%".format(float(i)*100/len(insts))+" Loss:{0:.2f}".format(error))
            sys.stdout.flush()
    if Verbose:
        sys.stdout.write("\n")
    return error

cdef void divide(REAL_t *vec, const float *alpha, const int *size):
    cdef int i
    for i from 0 <= i < size[0] by 1:
        vec[i] = vec[i]/alpha[0]
def cdivide(vec,alpha):
    cdef int size
    size  = len(vec)
    cdef float r = alpha
    divide(<REAL_t *>(np.PyArray_DATA(vec)),&r,&size)

    


def warp_gradient(A,B,inst,size,next_random,lr=0.01):
    #print B
    #print B[0]-B[9]
    cdef unsigned long long  c_next_random = next_random
    dA = np.zeros(size,dtype=REAL)
    dB = np.zeros(B.shape,dtype=REAL)

    x = np.sum(A[inst.features],axis=0)
    cdef REAL_t error = 0.
    cdef REAL_t clr = lr
    cdef int N,n_sample 
    cdef int neg_num = len(inst.negative_labels)
    cdef REAL_t norm
    cdef int cSize = size
    cdef REAL_t floats
    M = len(inst.sparse_labels)
    for l in inst.sparse_labels:
        s1 = cvdot(x,B[l],cSize)
        N = 1
        n_sample  = -1
        for k in range(neg_num):
            c_next_random = random_int32(&c_next_random)
            nl = inst.negative_labels[c_next_random%len(inst.negative_labels)]#randint(0,neg_num-1)]
            s2 = cvdot(x,B[nl],cSize)
            if s1 - s2<1:
                n_sample = nl
                N = k+1
                break
        if n_sample!=-1:
            L = crank(len(inst.negative_labels)/N)#+rev_crank(i)
            negL = -L
            error += (1+s2-s1)*L
            csaxpy(B[l]-B[n_sample],dA,L,cSize)

#             csaxpy(x,dB[l],L,cSize)
#             csaxpy(x,dB[n_sample],-L,cSize)
    for f in inst.features:
        csaxpy(dA,A[f],clr,cSize)
        norm = cnorm(A[f])
        if norm >1:
            cdivide(A[f],norm)
#     for i in range(len(B)):
#         csaxpy(dB[i],B[i],clr,cSize)
#         #B[i] += lr*dB[i]
#         norm =  cnorm(B[i])
#         if norm >1:
#             cdivide(B[i],norm)
#             B[i] /=norm
    return error,c_next_random


def save_to_text(matrix,output):
    shape = matrix.shape
    with open(output,'wb') as out:
        out.write("%d %d\n" % (shape))
        for row in matrix:
            x = " ".join(map(lambda x:"{0:.5}".format(x),row))
            out.write(x+"\n")

cdef inline unsigned long long random_int32(unsigned long long *next_random) nogil:
    next_random[0] = (next_random[0] * <unsigned long long>25214903917ULL + 11) & 281474976710655ULL
    return next_random[0]
def crand(sed):
    cdef unsigned long long csed = sed
    return random_int32(&csed)
def save2bin(mat,dct,fn):
    n,d  = mat.shape
    with open(fn,'w') as out:
        out.write("%d %d\n" % (n,d))
        for i in range(n):
            text = " ".join(map(str,mat[i]))
            out.write("%s %s\n" %(dct[i],text))
def normalize(mat):
    for v in mat:
        norm = np.linalg.norm(v)
        if norm >=1:
            v /= norm
def refine(labels,type_hierarchy,maxDepth=2,delim='/'):
    results = []
    labels = [labels[i] for i in range(len(labels))]
    pathes = [ type_hierarchy.get_type_path(labels[i]) for i in range(len(labels))]
    for i in range(len(labels)):
        for k in range(len(pathes[i])):
            if len(results)<k+1:
                results.append(pathes[i][k])
            elif results[-1]!=pathes[i][k]:
                break
    return results

def refine_data(A,B,data,tier):
    for inst in data:
        inst.sparse_labels = test(A,B,tier,inst)


def test(A,B,tier,inst,topk=3):
    cdef int cSize = B.shape[1]
    x = np.sum(A[inst.features],axis=0)

    scores = [cvdot(x,B[l],cSize) for l in inst.old_labels]
    ranks = argsort(scores,reverse=True)
    return refine([inst.old_labels[r] for r in ranks],tier)


In [12]:
in_dir= "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN"
a=MentionData('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_x_new.txt',
              "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_y.txt",
             in_dir+"/feature.txt",in_dir+"/type.txt")
label_bin  = Word2Vec.load_word2vec_format('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/labels.bin')
label_bin_hier  = Word2Vec.load_word2vec_format('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/labels_hier.bin')
#label_bin  = Word2Vec.load_word2vec_format('/Users/mayk/working/figer/baseline/PLE/Model/warp/embedding/BBN/inter_l2v_300.txt')

In [25]:
#np.random.seed(12)
size= 300
#A= np.random.normal(0,1, [len(a.feature2id),size]).astype(np.float32)#np.random.rand(len(a.feature2id),size).astype(np.float32)
A= np.random.uniform(-6/np.sqrt(size),6/np.sqrt(size), [len(a.feature2id),size]).astype(np.float32)#np.random.rand(len(a.feature2id),size).astype(np.float32)
B =np.asarray([label_bin[a.id2label[i]] for i in range(len(label_bin.vocab))],dtype=np.float32)#np.random.normal(0,1.0, [len(a.label2id),size]).astype(np.float32)
next_random = 1
normalize(A)
normalize(B)
for i in range(15): 
    ctrain(A,B,a.data,size,0.001,warp_gradient,it=i,Verbose=True)

save2bin(A,a.id2feature,'/Users/mayk/working/figer/baseline/PLE/Results/BBN/warp_A.bin')
save2bin(B,a.id2label,'/Users/mayk/working/figer/baseline/PLE/Results/BBN/warp_B.bin')

Iteration 0 trained 100% Loss:156172.92
Iteration 1 trained 100% Loss:105183.62
Iteration 2 trained 100% Loss:91517.77
Iteration 3 trained 100% Loss:83816.83
Iteration 4 trained 100% Loss:78126.23
Iteration 5 trained 100% Loss:74125.27
Iteration 6 trained 100% Loss:71152.59
Iteration 7 trained 100% Loss:68320.26
Iteration 8 trained 100% Loss:65867.26
Iteration 9 trained 100% Loss:63805.31
Iteration 10 trained 100% Loss:62444.55
Iteration 11 trained 100% Loss:60544.55
Iteration 12 trained 100% Loss:59716.10
Iteration 13 trained 100% Loss:58010.41
Iteration 14 trained 100% Loss:57262.91


In [ ]:
save2bin(A,a.id2feature,'/Users/mayk/working/figer/baseline/PLE/Results/warp_A.bin')
save2bin(B,a.id2label,'/Users/mayk/working/figer/baseline/PLE/Results/warp_B.bin')

In [42]:
def read_tid(fn,func=int,delim=" "):
    with open(fn,'r') as types:
        tid = dict()
        for ln in types:
            split = ln.rstrip().split(delim)
            tid[split[0]] = func(split[1])
    return tid
tid = read_tid('../../Intermediate/BBN/mention.txt',delim='\t')
wh=[tid[mid] for mid,name in  read_tid('../../Intermediate/BBN/mention_dict.txt',str,':').iteritems() if name == "White House"]

In [50]:
with open('../../Intermediate/BBN/relation.txt','r') as readin:
    for ln in readin:
        splited = ln.rstrip().split()
        right = int(splited[0])
        left = int(splited[1])
        if len(splited) < 3:
            a.data[right].sparse_labels = []
            a.data[left].sparse_labels = []
        else:
            labels = map(lambda x:a.label2id[x],splited[2:])
            full_labels = np.zeros(len(a.id2label),dtype=float)
            full_labels[labels]=1.
            a.data[right]=Instance(a.data[right].features,full_labels,labels)
            a.data[left]=Instance(a.data[left].features,full_labels,labels)


In [47]:
for idx in wh:
    labels = map(lambda x:a.label2id[x],['/ORGANIZATION','/ORGANIZATION/GOVERNMENT'])
    full_labels = np.zeros(len(a.id2label),dtype=float)
    full_labels[labels]=1.
    a.data[i]=Instance(a.data[i].features,full_labels,labels)

In [6]:

def filte_rare(data,labels):
    
    new_data = []
    ids = []
    for i in range(len(data)):
        skip = False
        for l in data[i].sparse_labels:
            if l in labels:
                skip =True
                break
        if not skip:
            ids.append(i)
            new_data.append(data[i])
    return ids,new_data
def keep_rare(data,labels):
    
    new_data = []
    for i in range(len(data)):
        skip = False
        for l in data[i].sparse_labels:
            if l in labels:
                skip =True
                break
        if skip:
            new_data.append(data[i])
    return new_data

In [25]:
rare_labels = set()
d = [0]*len(a.label2id)
for inst in a.data:
    for l in inst.sparse_labels:
        d[l]+=1
for i in range(len(d)):
    if d[i] < 500:
        print a.id2label[i]
        rare_labels.add(i)
ids,a.data= filte_rare(a.data,rare_labels)

/ANIMAL
/GAME
/ORGANIZATION/HOSPITAL
/FACILITY/ATTRACTION
/PRODUCT/WEAPON
/FACILITY/HIGHWAY_STREET
/LOCATION/CONTINENT
/ORGANIZATION/MUSEUM
/FACILITY/BUILDING
/FACILITY/BRIDGE
/WORK_OF_ART/PLAY
/LAW
/EVENT/HURRICANE
/PRODUCT/VEHICLE
/LOCATION/RIVER
/FACILITY/AIRPORT
/ORGANIZATION/HOTEL
/ORGANIZATION/RELIGIOUS
/LOCATION/LAKE_SEA_OCEAN


In [87]:
in_dir= "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN"
test=MentionData('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/test_x.txt',
              "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/test_y.txt",
             in_dir+"/feature.txt",in_dir+"/type.txt")

In [91]:
rare_labels = set()
d = [0]*len(a.label2id)
for inst in a.data:
    for l in inst.sparse_labels:
        d[l]+=1
for i in range(len(d)):
    if d[i] < 5000:
        rare_labels.add(i)
with open('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/mention.txt') as mentions:
    mention_dict = dict()
    for ln in mentions:
        splited = ln.rstrip().split()
        mention_dict[splited[0]] = splited[1]
ids,test.data = filte_rare(test.data,[l for l in range(len(test.label2id)) if l not in rare_labels])
origin_ids = [ ln.rstrip().split()[0] for ln in open('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/test_x.txt')]

In [92]:
with open('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/mention_type_zero.txt','w') as rare,open('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/test_x_zero.txt','w') as x:
    for i,inst in enumerate(test.data):
        x.write("%s\t%s\n" % (origin_ids[ids[i]],",".join(map(str,inst.features))))
        for l in inst.sparse_labels:
            rare.write("%s\t%d\t%d\n" %(mention_dict[origin_ids[ids[i]]],l,1))

In [13]:
tier= TypeHierarchy('../../Intermediate/BBN/supertype.txt',B.shape[0])
for inst in a.data:
    labels = set()
    for l in inst.sparse_labels:
        path= tier.get_type_path(l)
        labels.add(path[0])
    labels = list(labels)
    inst.sparse_labels = labels
#     full_labels = np.zeros(len(a.id2label),dtype=float)
#     full_labels[labels]=1.
#     a.data[i]=Instance(a.data[i].features,full_labels,labels)

In [19]:
translate = lambda x: map(lambda y:a.id2label[y],x)
translate(a.data[1009].sparse_labels)


['/PERSON']

In [23]:
label_bin_hier['/LOCATION']

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], dtype=float32)

In [24]:
b= map(lambda x:a.label2id[x],['/GPE','/GPE/CITY','/PERSON'])
refine(b,tier)

[18, 17]

In [25]:
b

[18, 17, 19]